In [3]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## NN related import and configurations

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import sampler
from torch.utils.data import DataLoader

import torchvision.datasets as dset
import torchvision.transforms as T

import timeit

## Data related

* Define ChunkSampler to split original train data into train part and validation part
* Define loaders

In [12]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset.
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from"""
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start
        
    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))
    
    def __len__(self):
        return self.num_samples
    
NUM_TRAIN = 49000
NUM_VAL = 1000

cifar10_train = dset.CIFAR10('../cs231n/datasets/', train=True, download=True, transform=T.ToTensor())
loader_train = DataLoader(cifar10_train, batch_size=64, sampler=ChunkSampler(NUM_TRAIN, 0))

cifar10_val = dset.CIFAR10('../cs231n/datasets/', train=True, download=True, transform=T.ToTensor())
# cifar10_val = cifar10_train # can do this?
loader_val = DataLoader(cifar10_val, batch_size=64, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

cifar10_test = dset.CIFAR10('../cs231n/datasets/', train=False, download=True, transform=T.ToTensor())
loader_test = DataLoader(cifar10_test, batch_size=64)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Basic configuration

In [21]:
dtype = torch.FloatTensor
if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor

print_every = 100

## First example model

In [22]:
import models

# model
example_model = models.ExampleNet().type(dtype)

# loss function
loss_fn = nn.CrossEntropyLoss().type(dtype)

# optimizer
optimizer = optim.Adam(example_model.parameters(), lr=1e-2)

### model sanity check

In [23]:
x = torch.randn(64, 3, 32, 32).type(dtype)
x_var = Variable(x, requires_grad=False)
ans = example_model(x_var)

np.array_equal(np.array(ans.size()), np.array([64, 10]))

True

In [31]:
%%timeit
torch.cuda.synchronize()
ans = example_model.type(dtype)(x_var)
torch.cuda.synchronize()

942 µs ± 2.36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
%%timeit
ans = example_model.type(torch.FloatTensor)(x_var.cpu())

18.7 ms ± 534 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
